
## Peer-graded Assignment: Построение baseline-решений
Настало время перейти к построению моделей! Давайте начнем с построения так называемых бейзлайнов - построим несколько моделей, которые в дальнейшем будем использовать в качестве первого приближения для будущей модели. Часто для решения подобных задач используются линейные модели, а также ансамбли, например, случайный лес или градиентный бустинг.

В этом задании вам предстоит построить несколько моделей и оценить их качество. Эти модели будут служить нам в качестве baseline-решений и пригодятся сразу для нескольких задач:

- Во-первых, на разработку baseline-модели не должно уходить много времени (это требование исходит из оценок затрат на проект в целом - большую часть времени все же нужно потратить на основное решение), процесс должен быть простым, на подавляющем большинстве этапов должны использоваться готовые протестированные инструменты. Все это приводит к тому, что baseline-модели - это дешевый способ грубо оценить потенциально возможного качества модели, при построении которого вероятность допущения ошибок относительно невелика.
- Во-вторых, использование моделей разного типа при построении baseline'ов позволяет на раннем этапе предположить, какие подходы являются наиболее перспективными и приоритизировать дальнейшие эксперименты.
- Наличие baseline-моделей позволяет оценить, какой прирост качества дают различные преобразования, усложнения, оптимизации и прочие активности, которые вы предпринимаете для построения финального решения.
- Наконец, если после построения сложного решения оценка его качества будет очень сильно отличаться от оценки качества baseline-моделей, то это будет хорошим поводом поискать в решении ошибки.  

Задание будет оцениваться на основе jupyter notebook'а, который вам нужно будет приложить в качестве решения. Убедитесь, что он содержит всю проделанную вами работу, код написан аккуратно и понятно, его легко читать, а также в тексте notebook'а присутствуют необходимые для понимания проделанной вами работы комментарии.

## Инструкции
Обучите 3 разные baseline-модели на полученных наборах данных и оцените их качество.

На прошлой неделе вы выбрали методику оценки качества моделей на основе кросс-валидации, а также основную и вспомогательные метрики. Оцените с их помощью получившуюся модель.

Обратите внимание, что под разными моделями понимаются именно разные алгоритмы классификации. Например, 2 модели, реализующие метод k ближайших соседей с разными k, будут считаться одним baseline-решением (хотя и с разными параметрами).

Напоминаем, что отложенная выборка (hold-out dataset) не должна использоваться для построения и оценки baseline-моделей!

Можно (но не обязательно) рассмотреть следующий набор алгоритмов:

- Линейная модель (например, реализация sklearn.linear_model.RidgeClassifier)
- Случайный лес (например, реализация sklearn.ensemble.RandomForestClassifier)
- Градиентный бустинг (например, реализация sklearn.ensemble.GradientBoostingClassifier)   

В качестве решения приложите получившийся jupyter notebook. Убедитесь, что в нем присутствуют:

- все baseline-модели, которые вы построили;
- качество всех построенных моделей оценено с помощью кросс-валидации, и это понятно из текста в jupyter notebook;
- все модели оценены с помощью основной и дополнительных метрик качества.

In [147]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as ss
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings("ignore")

In [148]:
x_data = pd.read_csv('orange_small_churn_data.train.txt')
y_data = pd.read_csv('orange_small_churn_labels.train.txt', header=None)
y_data.columns = ['Churn']
y_data['Churn'] = [0 if x == -1 else x for x in y_data['Churn']]

In [149]:
Na_columns = []

for column in x_data.columns: # Колонки, полностью состоящие из NaN
    if x_data[column].isna().sum() == x_data.shape[0]:
        x_data.drop(column, axis=1,inplace=True)
        Na_columns.append(column)
        
numeric = x_data.loc[:,:'Var190'].columns  
categorical = x_data.loc[:,'Var190':].columns        

# Колонки с полностью отсутствующими значениями

In [150]:
X, X_hold_out, y, y_hold_out = train_test_split(x_data, 
                                                    y_data,
                                                    test_size=0.1, shuffle=False, random_state=42) 

In [151]:
x_data.iloc[:,:190]

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var197,Var198,Var199,Var200,Var201,Var202,Var203,Var204,Var205,Var206
0,NaN,NaN,NaN,NaN,NaN,3052.0,NaN,NaN,NaN,NaN,...,0LaQ,UaKK0yW,I1sFbv_0IT,NaN,NaN,EkHG,9_Y1,k13i,09_Q,IYzP
1,NaN,NaN,NaN,NaN,NaN,1813.0,7.0,NaN,NaN,NaN,...,YFAj,Bnunsla,o64y9zI,DlISMzi,NaN,JDd6,9_Y1,FbIm,VpdQ,haYg
2,NaN,NaN,NaN,NaN,NaN,1953.0,7.0,NaN,NaN,NaN,...,TyGl,fhk21Ss,nQUveAzAF7,NaN,NaN,dnwD,9_Y1,mTeA,VpdQ,hAFG
3,NaN,NaN,NaN,NaN,NaN,1533.0,7.0,NaN,NaN,NaN,...,0Xwj,uoZk2Zj,LWyxgtXeJL,NaN,NaN,CwmB,F3hy,vzJD,VpdQ,IYzP
4,NaN,NaN,NaN,NaN,NaN,686.0,7.0,NaN,NaN,NaN,...,vSNn,kugYdIL,ZIXKpoNpqq,NaN,NaN,625Z,9_Y1,m_h1,sJzTlal,zm5i
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,NaN,NaN,NaN,NaN,NaN,938.0,7.0,NaN,NaN,NaN,...,JLbT,qMlD1gf,CdPgMY1Z52,IjmTVSM,smXZ,gMVu,9_Y1,RVjC,VpdQ,IYzP
39996,NaN,NaN,NaN,NaN,NaN,1750.0,7.0,NaN,NaN,NaN,...,lK27,Wtcr3WP,qzIoq9b,NaN,NaN,2qOt,9_Y1,m_h1,VpdQ,IYzP
39997,NaN,NaN,NaN,NaN,NaN,1204.0,7.0,NaN,NaN,NaN,...,487l,FZVtMxl,LH0kFz12FM,hDyVP2J,NaN,bSUY,9_Y1,MBhA,09_Q,6JmL
39998,NaN,NaN,NaN,NaN,NaN,8232.0,14.0,NaN,NaN,NaN,...,487l,gRinF0C,e1XhvTunuD,NaN,NaN,6wK7,9_Y1,7pVf,VpdQ,zm5i


In [152]:
for column in X[numeric]:
    median = X[column].dropna()[X[column].dropna() != 0].median()
    X[column].fillna(median, inplace=True)
    
X[categorical] = X[categorical].fillna('Na_cat') # Замена NaN категор.признаков соотв.категорией

In [153]:
le = LabelEncoder()
for column in X[categorical]:
    X[column] = le.fit_transform(X[column])

In [154]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [155]:
from sklearn.preprocessing import StandardScaler # Использование стандартизации

scaler = StandardScaler()

scaler.fit(X_train[numeric])

X_train[numeric] = scaler.transform(X_train[numeric])
X_test[numeric] = scaler.transform(X_test[numeric])

In [156]:
from sklearn.metrics import confusion_matrix, recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import (GridSearchCV,
                                     train_test_split,
                                     StratifiedKFold)

In [157]:
skf = StratifiedKFold(n_splits=5, random_state=17)

In [158]:
#Построение моделей

# Logistic Regression

logreg = LogisticRegression(penalty='l2', random_state=42) #l2 при мультиколлинеарности признаков
logreg_model = logreg.fit(X_train, y_train)
lr_predictions = logreg.predict(X_test)
lr_recall = recall_score(y_test, lr_predictions)
lr_f1_score= f1_score(y_test, lr_predictions)
lr_roc_auc = roc_auc_score(y_test, lr_predictions)

In [159]:
print(lr_recall, lr_f1_score, lr_roc_auc)

0.0 0.0 0.5


In [160]:
# Gradient Boosting

gbc = GradientBoostingClassifier(random_state=42)
gbc_model = gbc.fit(X_train, y_train)
gbc_predictions = gbc.predict(X_test)
gbc_recall = recall_score(y_test, gbc_predictions)
gbc_f1_score = f1_score(y_test, gbc_predictions)
gbc_roc_auc = roc_auc_score(y_test, gbc_predictions)

In [161]:
print(gbc_recall, gbc_f1_score, gbc_roc_auc)

0.016853932584269662 0.032846715328467155 0.5080519287883843


In [162]:
# Random Forest

random_forest = RandomForestClassifier(random_state=42)
rfc_model = random_forest.fit(X_train, y_train)
rfc_predictions = random_forest.predict(X_test)
random_forest.score(X_train, y_train)
rfc_recall = recall_score(y_test, rfc_predictions)
rfc_f1_score = f1_score(y_test, rfc_predictions)
rfc_roc_auc = roc_auc_score(y_test, rfc_predictions)

In [163]:
print(rfc_recall, rfc_f1_score, rfc_roc_auc)

0.0 0.0 0.5


In [164]:
gaussian = GaussianNB()
gauss_model = gaussian.fit(X_train, y_train)
gauss_predictions = gaussian.predict(X_test)
gauss_recall = recall_score(y_test, gauss_predictions)
gauss_f1_score= f1_score(y_test, gauss_predictions)
gauss_roc_auc = roc_auc_score(y_test, gauss_predictions)

In [165]:
gauss_recall

0.947565543071161

In [166]:
knn = KNeighborsClassifier()
knn_model = knn.fit(X_train, y_train)
knn_predictions = knn.predict(X_test)
knn_recall = recall_score(y_test, knn_predictions)
knn_f1_score= f1_score(y_test, knn_predictions)
knn_roc_auc = roc_auc_score(y_test, knn_predictions)

In [167]:
knn_recall

0.00749063670411985